# Anomaly detection in cpu based on box unix table

### Description
Notebook to generate an Isolation Model for tests (Unsupervising)

In [1]:
import numpy as np
import pandas as pd

In [2]:
import sys
print(sys.version)

3.8.10 (default, Mar 15 2022, 12:22:08) 
[GCC 9.4.0]


In [3]:
import os
ARTIFACTS_PATH = '../../artifacts/bigml/'
os.makedirs(ARTIFACTS_PATH, exist_ok=True) # Create path if not exists

### Save our BigML Username and Api Key to our environment to access the API

In [4]:
import os
os.environ['BIGML_USERNAME'] = "elizaglez"
os.environ['BIGML_API_KEY'] = "2064c7cbf0a35661eb33602d6ecd264aecdfd520"
os.environ['BIGML_CRED'] = "username=$BIGML_USERNAME;api_key=$BIGML_API_KEY"
os.environ['BIGML_PROJECT'] = "project/59ef803eaf447f0b1d001156"

### Creat our main API object that all the main functions will utilize.

In [5]:
from bigml.api import BigML
api = BigML(project="project/59ef803eaf447f0b1d001156")

In [6]:
data_source = api.create_source("../../data/cpu/cpu_optimo.csv")
api.ok(data_source)

True

In [7]:
data = api.create_dataset(data_source)
api.ok(data)

True

## Test-Train Split

In [8]:
train_dataset = api.create_dataset(data,
                                   {"name": "cpu",
                                    "sample_rate": 0.8, "seed": "my seed",
                                    "tags": ["cpu_anomaly"]})
api.ok(train_dataset)

True

### Isolation Forest (Anomaly)

In [9]:
model_name = "anomaly_bigml"
desc = "bigml anomaly detection for cpu"
model_out_file_name = model_name + ".json"
model_out_path = os.path.join(ARTIFACTS_PATH, model_out_file_name)

In [10]:
anomaly = api.create_anomaly(train_dataset, {
    "name" : model_name,
    "tags" : [model_name]})
api.ok(anomaly)

True

In [11]:
api.export_last(model_name,
                resource_type="anomaly",
                filename=model_out_path)


'../../artifacts/bigml//anomaly_bigml.json'

In [12]:
input_data = {"cpuSys":0.0,"memUsed":2.1059371008E10,"memBuff":1.75958016E9,"memCach":7.392792576E9,"memFree":3.354673152E9,"swapUsed":3321856,"swapFree":3.4181300224E10,"systemCsw":8080.817,"loadFif":0.89,"proTotal":1541,"proPid":28454,"eventdate.hour":7,"eventdate.minute": 44}
# assigning an anomaly score to it
anomaly_score = api.create_anomaly_score(anomaly, input_data)
print(anomaly_score["object"]['score'])

0.64916
